In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/Thesis_work/DGCNN/dgcnn/pytorch

/content/gdrive/MyDrive/Thesis_work/DGCNN/dgcnn/pytorch


In [ ]:
!pip install open3d

     |████████████████████████████████| 395.5 MB 29 kB/s 
     |████████████████████████████████| 596 kB 59.1 MB/s 
     |████████████████████████████████| 8.5 MB 72.7 MB/s 
     |████████████████████████████████| 1.1 MB 60.4 MB/s 
     |████████████████████████████████| 4.3 MB 56.6 MB/s 
     |████████████████████████████████| 61 kB 10.0 MB/s 
     |████████████████████████████████| 428 kB 77.2 MB/s 
     |████████████████████████████████| 395 kB 71.9 MB/s 
     |████████████████████████████████| 53 kB 2.7 MB/s 
     |████████████████████████████████| 130 kB 78.4 MB/s 
     |████████████████████████████████| 79 kB 10.4 MB/s 
  Attempting uninstall: tornado
    Found existing installation: tornado 5.1.1
    Uninstalling tornado-5.1.1:
      Successfully uninstalled tornado-5.1.1
  Attempting uninstall: pygments
    Found existing installation: Pygments 2.6.1
    Uninstalling Pygments-2.6.1:
      Successfully uninstalled Pygments-2.6.1
  Attempting uninstall: jupyter-client
    Found e

In [ ]:
from __future__ import print_function
import os
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from data import ModelNet40
from model import PointNet, DGCNN
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from util import cal_loss, IOStream
import sklearn.metrics as metrics
import numpy as np
import math
import random
import open3d as o3d

import sys
import copy
import math


In [ ]:
!pip install path.py;
from path import Path

In [ ]:
path = Path("/content/gdrive/MyDrive/Thesis_work/Datasets/RGBD Dataset")
random.seed = 49
print(path)

/content/gdrive/MyDrive/Thesis_work/Datasets/RGBD Dataset


In [ ]:
folders = [dir for dir in sorted(os.listdir(path)) if os.path.isdir(path/dir)]
classes = {folder: i for i, folder in enumerate(folders)};
classes

{'apple': 0,
 'binder': 1,
 'calculator': 2,
 'cereal_box': 3,
 'coffee_mug': 4,
 'dry_battery': 5,
 'flashlight': 6,
 'food_can': 7,
 'food_cup': 8,
 'garlic': 9,
 'greens': 10,
 'keyboard': 11,
 'kleenex': 12,
 'lightbulb': 13,
 'lime': 14,
 'mushroom': 15,
 'notebook': 16,
 'onion': 17,
 'pear': 18,
 'pitcher': 19,
 'potato': 20,
 'soda_can': 21,
 'sponge': 22,
 'toothpaste': 23,
 'water_bottle': 24}

# pcd file read function

In [ ]:
def read_pcd(file):
    pcd = o3d.io.read_point_cloud(file, format="xyz")
    points = np.asarray(pcd.points)  
    return points

In [ ]:
class PointSampler(object):
    def __init__(self, output_size):
        assert isinstance(output_size, int)
        self.output_size = output_size
    
    def sample_point(self, pt1, pt2, pt3):
        # barycentric coordinates on a triangle
        # https://mathworld.wolfram.com/BarycentricCoordinates.html
        s, t = sorted([random.random(), random.random()])
        f = lambda i: s * pt1[i] + (t-s)*pt2[i] + (1-t)*pt3[i]
        return (f(0), f(1), f(2))
          
    def __call__(self, mesh):
        verts = mesh
        verts = np.array(verts)
        sampled_points = np.array(random.choices(verts, 
                                      k=self.output_size))
        
        return sampled_points

In [ ]:
class ToTensor(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape)==2

        return torch.from_numpy(pointcloud)

In [ ]:
class Normalize(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape)==2
        
        norm_pointcloud = pointcloud - np.mean(pointcloud, axis=0) 
        norm_pointcloud /= np.max(np.linalg.norm(norm_pointcloud, axis=1))

        return  norm_pointcloud

In [ ]:
def default_transforms():
    return transforms.Compose([
                                PointSampler(1024),
                                Normalize(),
                                ToTensor()
                              ])

In [ ]:
train_transforms = transforms.Compose([
                    PointSampler(1024),
                    Normalize(),
                    ToTensor()
                    ])

In [ ]:
class PointCloudData(Dataset):
    def __init__(self, files, classes, transform=train_transforms):
        self.files = files
        self.classes = classes
        self.transform = transform
    
    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        pcd_path = self.files[idx]['pcd_path']
        category = self.files[idx]['category']
        return {'pointcloud': self.transform((read_pcd(pcd_path))), 
                'category': self.classes[category]}

In [ ]:
def ds(root_dir):
    folders = [dir for dir in sorted(os.listdir(root_dir))]
    classes = {folder: i for i, folder in enumerate(folders)}
    train_files = []
    valid_files = []
    for category in classes.keys():
        train_folders = [dir for dir in sorted(os.listdir(root_dir+"/"+category))]
        valid_folder = train_folders.pop()
        for folder in train_folders:
            new_dir = root_dir/Path(category)/folder
            dir = sorted(os.listdir(new_dir))[::4]
            for file in dir:
                if file.endswith('.pcd'):
                    sample = {}
                    sample['pcd_path'] = new_dir/file
                    sample['category'] = category
                    train_files.append(sample)
        new_dir = root_dir/Path(category)/valid_folder
        valid_dir = root_dir/Path(category)/valid_folder
        valid_dir = sorted(os.listdir(valid_dir))[::4]
        for file in valid_dir:
            if file.endswith('.pcd'):
                sample = {}
                sample['pcd_path'] = new_dir/file
                sample['category'] = category
                valid_files.append(sample)
    return train_files, valid_files

In [ ]:
train_files, valid_files = ds(path)
train_ds = PointCloudData(train_files, classes, transform=train_transforms)
valid_ds = PointCloudData(valid_files, classes, transform=train_transforms)

In [ ]:
device = torch.device('cuda')
print(device)

cuda


In [ ]:
print("Reading all train PCD files \nSamples read:",end=" ")
X = []
for i in range(len(train_ds)):
    if i%100==0:
        print(i,end="-")
    X.append(train_ds[i])   

Reading all train PCD files 
Samples read: 0-100-200-300-400-500-600-700-800-900-1000-1100-1200-1300-1400-1500-1600-1700-1800-1900-2000-2100-2200-2300-2400-2500-2600-2700-2800-2900-3000-3100-3200-3300-3400-3500-3600-3700-3800-3900-4000-4100-4200-4300-4400-4500-4600-4700-4800-4900-5000-5100-5200-5300-5400-5500-5600-5700-5800-5900-6000-6100-6200-6300-6400-6500-6600-6700-6800-6900-7000-7100-7200-7300-7400-7500-7600-7700-7800-7900-8000-8100-8200-8300-8400-8500-8600-8700-8800-8900-9000-9100-9200-9300-9400-9500-9600-9700-9800-9900-10000-10100-10200-10300-10400-10500-

In [ ]:
print("Reading all valid PCD files \nSamples read:",end=" ")
X_val = []
for i in range(len(valid_ds)):
    if i%100==0:
        print(i,end="-")
    X_val.append(valid_ds[i])

Reading all valid PCD files 
Samples read: 0-100-200-300-400-500-600-700-800-900-1000-1100-1200-1300-1400-1500-1600-1700-1800-1900-2000-2100-2200-2300-2400-2500-

In [ ]:
train_loader = DataLoader(dataset=X, batch_size=32, shuffle=True, drop_last=True)
test_loader = DataLoader(dataset=X_val, batch_size=16, shuffle=False, drop_last=False)

In [ ]:
def knn(x, k):
    inner = -2*torch.matmul(x.transpose(2, 1), x)
    xx = torch.sum(x**2, dim=1, keepdim=True)
    pairwise_distance = -xx - inner - xx.transpose(2, 1)
 
    idx = pairwise_distance.topk(k=k, dim=-1)[1]   # (batch_size, num_points, k)
    return idx


def get_graph_feature(x, k=20, idx=None):
    batch_size = x.size(0)
    num_points = x.size(2)
    x = x.view(batch_size, -1, num_points)
    if idx is None:
        idx = knn(x, k=k)   # (batch_size, num_points, k)
    device = torch.device('cuda')

    idx_base = torch.arange(0, batch_size, device=device).view(-1, 1, 1)*num_points

    idx = idx + idx_base

    idx = idx.view(-1)
 
    _, num_dims, _ = x.size()

    x = x.transpose(2, 1).contiguous()   # (batch_size, num_points, num_dims)  -> (batch_size*num_points, num_dims) #   batch_size * num_points * k + range(0, batch_size*num_points)
    feature = x.view(batch_size*num_points, -1)[idx, :]
    feature = feature.view(batch_size, num_points, k, num_dims) 
    x = x.view(batch_size, num_points, 1, num_dims).repeat(1, 1, k, 1)
    
    feature = torch.cat((feature-x, x), dim=3).permute(0, 3, 1, 2).contiguous()
  
    return feature

class DGCNN(nn.Module):
    def __init__(self, output_channels=30):
        super(DGCNN, self).__init__()
        self.k = 20
        
        self.bn1 = nn.BatchNorm2d(64)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
        self.bn4 = nn.BatchNorm2d(256)
        self.bn5 = nn.BatchNorm1d(1024)

        self.conv1 = nn.Sequential(nn.Conv2d(6, 64, kernel_size=1, bias=False),
                                   self.bn1,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv2 = nn.Sequential(nn.Conv2d(64*2, 64, kernel_size=1, bias=False),
                                   self.bn2,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv3 = nn.Sequential(nn.Conv2d(64*2, 128, kernel_size=1, bias=False),
                                   self.bn3,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv4 = nn.Sequential(nn.Conv2d(128*2, 256, kernel_size=1, bias=False),
                                   self.bn4,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv5 = nn.Sequential(nn.Conv1d(512, 1024, kernel_size=1, bias=False),
                                   self.bn5,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.linear1 = nn.Linear(1024*2, 512, bias=False)
        self.bn6 = nn.BatchNorm1d(512)
        self.dp1 = nn.Dropout(p=0.6)
        self.linear2 = nn.Linear(512, 256)
        self.bn7 = nn.BatchNorm1d(256)
        self.dp2 = nn.Dropout(p=0.6)
        self.linear3 = nn.Linear(256, output_channels)

    def forward(self, x):
        batch_size = x.size(0)
        x = get_graph_feature(x, k=self.k)
        x = self.conv1(x)
        x1 = x.max(dim=-1, keepdim=False)[0]

        x = get_graph_feature(x1, k=self.k)
        x = self.conv2(x)
        x2 = x.max(dim=-1, keepdim=False)[0]

        x = get_graph_feature(x2, k=self.k)
        x = self.conv3(x)
        x3 = x.max(dim=-1, keepdim=False)[0]
      
        x = get_graph_feature(x3, k=self.k)
        x = self.conv4(x)
        x4 = x.max(dim=-1, keepdim=False)[0]
      
        x = torch.cat((x1, x2, x3, x4), dim=1)

        x = self.conv5(x)
  
        x1 = F.adaptive_max_pool1d(x, 1).view(batch_size, -1)
        x2 = F.adaptive_avg_pool1d(x, 1).view(batch_size, -1)
        x = torch.cat((x1, x2), 1)
        
        x = F.leaky_relu(self.bn6(self.linear1(x)), negative_slope=0.2)
        x = self.dp1(x)
        x = F.leaky_relu(self.bn7(self.linear2(x)), negative_slope=0.2)
        x = self.dp2(x)
        x = self.linear3(x)
        return x

In [ ]:
def train(epochs,model,train_loader,test_loader, sgd, lr, momentum):
    
    #Try to load models
    # if args.model == 'pointnet':
    #     model = PointNet(args).to(device)
    # elif args.model == 'dgcnn':
    #     model = DGCNN(args).to(device)
    # else:
    #     raise Exception("Not implemented")
    # print(str(model))
    
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    
    PATH = "dgcnn.pth"
#     checkpoint = torch.load(PATH)
#     model.load_state_dict(checkpoint["model_state_dict"])

    if sgd:
        print("Use SGD")
        opt = optim.SGD(model.parameters(), lr=lr*100, momentum=momentum, weight_decay=1e-4)
    else:
        print("Use Adam")
        opt = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)

    scheduler = CosineAnnealingLR(opt, epochs, eta_min=0.0001)
    
    criterion = cal_loss

    best_test_acc = 0
    for epoch in range(epochs):
        opt.step()
        scheduler.step()
        ####################
        # Train
        ####################
        train_loss = 0.0
        count = 0.0
        model.train()
        train_pred = []
        train_true = []
        for data in train_loader:
            label = data['category'].to(device)
            data  = data['pointcloud'].to(device).float() 
            data = data.permute(0, 2, 1)
            batch_size = data.size()[0]
            opt.zero_grad()
            logits = model(data)
            loss = criterion(logits, label)
            loss.backward()
            opt.step()
            preds = logits.max(dim=1)[1]
            count += batch_size
            train_loss += loss.item() * batch_size
            train_true.append(label.cpu().numpy())
            train_pred.append(preds.detach().cpu().numpy())
        train_true = np.concatenate(train_true)
        train_pred = np.concatenate(train_pred)
        # torch.save(model.state_dict(), "save_"+str(epoch)+".pth")
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            'loss': loss,
            }, "dgcnn_"+str(epoch)+".pth")
        outstr = 'Train %d, loss: %.6f, train acc: %.6f, train avg acc: %.6f' % (epoch,
                                                                                 train_loss*1.0/count,
                                                                                 metrics.accuracy_score(
                                                                                     train_true, train_pred),
                                                                                 metrics.balanced_accuracy_score(
                                                                                     train_true, train_pred))
        print(outstr)

        ####################
        # Test
        ####################
        test_loss = 0.0
        count = 0.0
        model.eval()
        test_pred = []
        test_true = []
        for data in test_loader:
            label = data['category'].to(device).squeeze()
            data= data['pointcloud'].to(device).float() 
            data = data.permute(0, 2, 1)
            batch_size = data.size()[0]
            logits = model(data)
            loss = criterion(logits, label)
            preds = logits.max(dim=1)[1]
            count += batch_size
            test_loss += loss.item() * batch_size
            test_true.append(label.cpu().numpy())
            test_pred.append(preds.detach().cpu().numpy())
        test_true = np.concatenate(test_true)
        test_pred = np.concatenate(test_pred)
        test_acc = metrics.accuracy_score(test_true, test_pred)
        avg_per_class_acc = metrics.balanced_accuracy_score(test_true, test_pred)
        outstr = 'Test %d, loss: %.6f, test acc: %.6f, test avg acc: %.6f' % (epoch,
                                                                              test_loss*1.0/count,
                                                                              test_acc,
                                                                              avg_per_class_acc)
        print(outstr)
        if test_acc >= best_test_acc:
            best_test_acc = test_acc
            #torch.save(model.state_dict(), 'checkpoints/%s/models/model.t7' % args.exp_name)
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': opt.state_dict(),
                'loss': loss,
                }, "dgcnn_rgbd_"+str(epoch)+"_"+str(best_test_acc+".pth"))

In [ ]:
def train(epochs,train_loader,test_loader, sgd, lr, momentum):
    # train_loader = DataLoader(ModelNet40(partition='train', num_points=args.num_points), num_workers=8,
    #                           batch_size=args.batch_size, shuffle=True, drop_last=True)
    # test_loader = DataLoader(ModelNet40(partition='test', num_points=args.num_points), num_workers=8,
    #                          batch_size=args.test_batch_size, shuffle=True, drop_last=False)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    #Try to load models
    # if args.model == 'pointnet':
    #     model = PointNet(args).to(device)
    # elif args.model == 'dgcnn':
    #     model = DGCNN(args).to(device)
    # else:
    #     raise Exception("Not implemented")
    # print(str(model))
    model = DGCNN().to(device)
    model = nn.DataParallel(model)
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    
    PATH = "/content/gdrive/MyDrive/Thesis_work/DGCNN/dgcnn/pytorch/dgcnn_13_86.pth"
    checkpoint = torch.load(PATH)
    model.load_state_dict(checkpoint["model_state_dict"])

    if sgd:
        print("Use SGD")
        opt = optim.SGD(model.parameters(), lr=lr*100, momentum=momentum, weight_decay=1e-4)
    else:
        print("Use Adam")
        opt = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)

    scheduler = CosineAnnealingLR(opt, epochs, eta_min=0.0001)
    
    criterion = cal_loss

    best_test_acc = 0
    for epoch in range(epochs):
        opt.step()
        scheduler.step()
        ####################
        # Train
        ####################
        train_loss = 0.0
        count = 0.0
        model.train()
        train_pred = []
        train_true = []
        for data in train_loader:
            data, label = data['pointcloud'].to(device=device, dtype=torch.float), data['category'].to(device).squeeze()
            data = data.permute(0, 2, 1)
            batch_size = data.size()[0]
            opt.zero_grad()
            logits = model(data)
            loss = criterion(logits, label)
            loss.backward()
            opt.step()
            preds = logits.max(dim=1)[1]
            count += batch_size
            train_loss += loss.item() * batch_size
            train_true.append(label.cpu().numpy())
            train_pred.append(preds.detach().cpu().numpy())
        train_true = np.concatenate(train_true)
        train_pred = np.concatenate(train_pred)
        # torch.save(model.state_dict(), "save_"+str(epoch)+".pth")
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            'loss': loss,
            }, "dgcnn_"+str(epoch+14)+".pth")
        outstr = 'Train %d, loss: %.6f, train acc: %.6f, train avg acc: %.6f' % (epoch,
                                                                                 train_loss*1.0/count,
                                                                                 metrics.accuracy_score(
                                                                                     train_true, train_pred),
                                                                                 metrics.balanced_accuracy_score(
                                                                                     train_true, train_pred))
        print(outstr)

        ####################
        # Test
        ####################
        test_loss = 0.0
        count = 0.0
        model.eval()
        test_pred = []
        test_true = []
        for data in test_loader:
            data, label = data['pointcloud'].to(device=device, dtype=torch.float), data['category'].to(device).squeeze()
            data = data.permute(0, 2, 1)
            batch_size = data.size()[0]
            logits = model(data)
            loss = criterion(logits, label)
            preds = logits.max(dim=1)[1]
            count += batch_size
            test_loss += loss.item() * batch_size
            test_true.append(label.cpu().numpy())
            test_pred.append(preds.detach().cpu().numpy())
        test_true = np.concatenate(test_true)
        test_pred = np.concatenate(test_pred)
        test_acc = metrics.accuracy_score(test_true, test_pred)
        avg_per_class_acc = metrics.balanced_accuracy_score(test_true, test_pred)
        outstr = 'Test %d, loss: %.6f, test acc: %.6f, test avg acc: %.6f' % (epoch,
                                                                              test_loss*1.0/count,
                                                                              test_acc,
                                                                              avg_per_class_acc)
        print(outstr)
        if test_acc >= best_test_acc:
            best_test_acc = test_acc
            #torch.save(model.state_dict(), 'checkpoints/%s/models/model.t7' % args.exp_name)
            # torch.save({
            # 'epoch': epoch,
            # 'model_state_dict': model.state_dict(),
            # 'optimizer_state_dict': opt.state_dict(),
            # 'loss': loss,
            # }, "rgb_dgcnn_" + str(epoch+1)+".pth")
# def test(args, io):
#     test_loader = DataLoader(ModelNet40(partition='test', num_points=args.num_points),
#                              batch_size=args.test_batch_size, shuffle=True, drop_last=False)

#     device = torch.device("cuda" if args.cuda else "cpu")

#     #Try to load models
#     model = DGCNN(args).to(device)
#     model = nn.DataParallel(model)
#     model.load_state_dict(torch.load(args.model_path))
#     model = model.eval()
#     test_acc = 0.0
#     count = 0.0
#     test_true = []
#     test_pred = []
#     for data, label in test_loader:

#         data, label = data.to(device), label.to(device).squeeze()
#         data = data.permute(0, 2, 1)
#         batch_size = data.size()[0]
#         logits = model(data)
#         preds = logits.max(dim=1)[1]
#         test_true.append(label.cpu().numpy())
#         test_pred.append(preds.detach().cpu().numpy())
#     test_true = np.concatenate(test_true)
#     test_pred = np.concatenate(test_pred)
#     test_acc = metrics.accuracy_score(test_true, test_pred)
#     avg_per_class_acc = metrics.balanced_accuracy_score(test_true, test_pred)
#     outstr = 'Test :: test acc: %.6f, test avg acc: %.6f'%(test_acc, avg_per_class_acc)
#     io.cprint(outstr)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DGCNN().to(device)
model = nn.DataParallel(model)

train(20, train_loader, test_loader, False, 0.0001, 0.9)

Let's use 1 GPUs!
Use Adam


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


Train 0, loss: 2.086810, train acc: 0.669751, train avg acc: 0.543355
Test 0, loss: 2.018848, test acc: 0.690846, test avg acc: 0.687120


UFuncTypeError: ignored

# Extracting Feature Vector

In [ ]:
path = 

In [ ]:
unseen_test_ds = PointCloudData(path, folder='test', transform=train_transforms)

In [ ]:
unseen_test_loader = DataLoader(dataset=unseen_test_ds, shuffle=False)

In [ ]:
def knn(x, k):
    inner = -2*torch.matmul(x.transpose(2, 1), x)
    xx = torch.sum(x**2, dim=1, keepdim=True)
    pairwise_distance = -xx - inner - xx.transpose(2, 1)
 
    idx = pairwise_distance.topk(k=k, dim=-1)[1]   # (batch_size, num_points, k)
    return idx


def get_graph_feature(x, k=20, idx=None):
    batch_size = x.size(0)
    num_points = x.size(2)
    x = x.view(batch_size, -1, num_points)
    if idx is None:
        idx = knn(x, k=k)   # (batch_size, num_points, k)
    device = torch.device('cuda')

    idx_base = torch.arange(0, batch_size, device=device).view(-1, 1, 1)*num_points

    idx = idx + idx_base

    idx = idx.view(-1)
 
    _, num_dims, _ = x.size()

    x = x.transpose(2, 1).contiguous()   # (batch_size, num_points, num_dims)  -> (batch_size*num_points, num_dims) #   batch_size * num_points * k + range(0, batch_size*num_points)
    feature = x.view(batch_size*num_points, -1)[idx, :]
    feature = feature.view(batch_size, num_points, k, num_dims) 
    x = x.view(batch_size, num_points, 1, num_dims).repeat(1, 1, k, 1)
    
    feature = torch.cat((feature-x, x), dim=3).permute(0, 3, 1, 2).contiguous()
  
    return feature

class DGCNN(nn.Module):
    def __init__(self, output_channels=30):
        super(DGCNN, self).__init__()
        self.k = 20
        
        self.bn1 = nn.BatchNorm2d(64)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
        self.bn4 = nn.BatchNorm2d(256)
        self.bn5 = nn.BatchNorm1d(1024)

        self.conv1 = nn.Sequential(nn.Conv2d(6, 64, kernel_size=1, bias=False),
                                   self.bn1,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv2 = nn.Sequential(nn.Conv2d(64*2, 64, kernel_size=1, bias=False),
                                   self.bn2,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv3 = nn.Sequential(nn.Conv2d(64*2, 128, kernel_size=1, bias=False),
                                   self.bn3,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv4 = nn.Sequential(nn.Conv2d(128*2, 256, kernel_size=1, bias=False),
                                   self.bn4,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv5 = nn.Sequential(nn.Conv1d(512, 1024, kernel_size=1, bias=False),
                                   self.bn5,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.linear1 = nn.Linear(1024*2, 512, bias=False)
        self.bn6 = nn.BatchNorm1d(512)
        self.dp1 = nn.Dropout(p=0.6)
        self.linear2 = nn.Linear(512, 256)
        self.bn7 = nn.BatchNorm1d(256)
        self.dp2 = nn.Dropout(p=0.6)
        self.linear3 = nn.Linear(256, output_channels)

    def forward(self, x):
        batch_size = x.size(0)
        x = get_graph_feature(x, k=self.k)
        x = self.conv1(x)
        x1 = x.max(dim=-1, keepdim=False)[0]

        x = get_graph_feature(x1, k=self.k)
        x = self.conv2(x)
        x2 = x.max(dim=-1, keepdim=False)[0]

        x = get_graph_feature(x2, k=self.k)
        x = self.conv3(x)
        x3 = x.max(dim=-1, keepdim=False)[0]
      
        x = get_graph_feature(x3, k=self.k)
        x = self.conv4(x)
        x4 = x.max(dim=-1, keepdim=False)[0]
      
        x = torch.cat((x1, x2, x3, x4), dim=1)

        x = self.conv5(x)
        
        x1 = F.adaptive_max_pool1d(x, 1).view(batch_size, -1)
        x2 = F.adaptive_avg_pool1d(x, 1).view(batch_size, -1)
        x = torch.cat((x1, x2), 1)
        y = x
        
        x = F.leaky_relu(self.bn6(self.linear1(x)), negative_slope=0.2)
        x = self.dp1(x)
        x = F.leaky_relu(self.bn7(self.linear2(x)), negative_slope=0.2)
        x = self.dp2(x)
        x = self.linear3(x)
        return x, y

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model = DGCNN().to(device)

In [ ]:
model = nn.DataParallel(model)

In [ ]:
PATH = "/content/gdrive/MyDrive/Thesis_work/DGCNN/dgcnn/pytorch/dgcnn_13_86.pth"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint["model_state_dict"])

In [ ]:
import pandas as pd

In [ ]:
def feedforward(train_loader):
    model.eval()
    df = pd.DataFrame(columns=[i for i in range(2049)])
    #feature_vector = torch.zeros([1, 1024], dtype=torch.int32).to(device) 
    for i, data in enumerate(train_loader, 0):
        data, labels = data['pointcloud'].to(device=device, dtype=torch.float), data['category'].to(device).squeeze()
        data = data.permute(0, 2, 1)
        batch_size = data.size()[0]
        output, feature_vector = model(data)
        # inputs, labels = data['pointcloud'].to(device).float(), data['category'].to(device)
        # feature_vector, outputs = model(inputs.transpose(1,2))
        feature_vector_np = feature_vector.cpu().detach().numpy()
        label_np = labels.cpu().detach().numpy()
        # print(len(label_np), feature_vector_np.shape)
        df.loc[i] = np.append(label_np, feature_vector_np.flatten())
        #feature_vector = torch.cat((feature_vector, vector), dim=0)
        if i % 100 == 0:
            print(f"DataFrame Size {df.shape}")
            print(df.head(i))
    df.to_csv('feature_vector_test_model30.csv')
    print("Done")

In [ ]:
feedforward(train_loader)